# Geospatial Analysis of World Development Indicators

## Step 1: Import Data and Libraries

In [216]:
# import libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import folium
import json
import geopandas as gpd
from os.path import join
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster

In [217]:
# setup

%matplotlib inline
sns.set_style('whitegrid')

In [218]:
# create path

path = r'../Global-Development-Trends/Data_Extract_From_World_Development_Indicators/Data'

In [219]:
# import data

df_clean = pd.read_csv('Clean_Data.csv')

In [220]:
# Display basic info

print(df_clean.info())
print(df_clean.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4737 entries, 0 to 4736
Data columns (total 9 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   Country Name                                      4737 non-null   object 
 1   Country Code                                      4737 non-null   object 
 2   Year                                              4737 non-null   int64  
 3   GDP per capita (current US$)                      4600 non-null   float64
 4   Individuals using the Internet (% of population)  3365 non-null   float64
 5   Life expectancy at birth, total (years)           4505 non-null   float64
 6   GDP_growth                                        4384 non-null   float64
 7   LE_change                                         4240 non-null   float64
 8   Internet_growth                                   3535 non-null   float64
dtypes: float64(6), int64

## Step 2: Get GeoJSON Data

In [221]:
# Using a world countries GeoJSON file

world_geojson_url = 'https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json'

# Download the GeoJSON data
response = requests.get(world_geojson_url)
world_geojson = response.json()

# Let's check the structure of the GeoJSON
print("GeoJSON properties:", list(world_geojson['features'][0]['properties'].keys()))

GeoJSON properties: ['name']


## Step 3: Data Wrangling and Cleaning 

In [222]:
# Display basic information about the dataset

print("=== DATASET OVERVIEW ===")
print(f"Shape: {df_clean.shape}")
print(f"Columns: {list(df_clean.columns)}")
print(f"Years range: {df_clean['Year'].min()} - {df_clean['Year'].max()}")
print(f"Number of countries: {df_clean['Country Name'].nunique()}")
print("\n")

=== DATASET OVERVIEW ===
Shape: (4737, 9)
Columns: ['Country Name', 'Country Code', 'Year', 'GDP per capita (current US$)', 'Individuals using the Internet (% of population)', 'Life expectancy at birth, total (years)', 'GDP_growth', 'LE_change', 'Internet_growth']
Years range: 2000 - 2024
Number of countries: 265




In [223]:
# check for missing values

print("=== MISSING VALUES ANALYSIS ===")
missing_values = df_clean.isnull().sum()
missing_percentage = (df_clean.isnull().sum() / len(df_clean)) * 100
missing_df = pd.DataFrame({
    'Missing Values': missing_values,
    'Percentage': missing_percentage
})
print(missing_df[missing_df['Missing Values'] > 0])
print("\n")

=== MISSING VALUES ANALYSIS ===
                                                  Missing Values  Percentage
GDP per capita (current US$)                                 137    2.892126
Individuals using the Internet (% of population)            1372   28.963479
Life expectancy at birth, total (years)                      232    4.897615
GDP_growth                                                   353    7.451974
LE_change                                                    497   10.491872
Internet_growth                                             1202   25.374710




In [224]:
# Handle missing values strategically

# 1. Handle missing GDP values - we'll use forward fill within each country
df_sorted = df_clean.sort_values(['Country Name', 'Year'])
df_clean = df_sorted.copy()

In [225]:
# Forward fill for GDP within each country group

df_clean['GDP per capita (current US$)'] = df_clean.groupby('Country Name')['GDP per capita (current US$)'].ffill()


In [226]:
# 2. Handle internet usage missing values - use interpolation

df_clean['Individuals using the Internet (% of population)'] = df_clean.groupby('Country Name')['Individuals using the Internet (% of population)'].transform(
    lambda x: x.interpolate(method='linear', limit_direction='both') if x.notna().sum() > 1 else x)

In [227]:
# 3. For countries with no internet data, we'll drop those rows for internet-related analysis

df_internet = df_clean.dropna(subset=['Individuals using the Internet (% of population)'])

In [228]:
# 4. Check for extreme values in GDP

print("=== EXTREME VALUES ANALYSIS ===")
gdp_stats = df_clean['GDP per capita (current US$)'].describe()
print("GDP Statistics:")
print(gdp_stats)
print("\n")

=== EXTREME VALUES ANALYSIS ===
GDP Statistics:
count      4646.000000
mean      14696.631826
std       23262.463513
min         109.593814
25%        1537.845680
50%        5060.321714
75%       18634.629914
max      256580.515123
Name: GDP per capita (current US$), dtype: float64




In [229]:
# Identify potential outliers (beyond 3 standard deviations)

gdp_mean = df_clean['GDP per capita (current US$)'].mean()
gdp_std = df_clean['GDP per capita (current US$)'].std()
outlier_threshold = gdp_mean + 3 * gdp_std
outliers = df_clean[df_clean['GDP per capita (current US$)'] > outlier_threshold]
print(f"Potential GDP outliers (>{outlier_threshold:.2f}): {len(outliers)} countries")
print(outliers[['Country Name', 'Year', 'GDP per capita (current US$)']].head(10))
print("\n")


Potential GDP outliers (>84484.02): 92 countries
    Country Name  Year  GDP per capita (current US$)
435      Bermuda  2013                 101704.407462
436      Bermuda  2014                 100961.576603
437      Bermuda  2015                 104855.366822
438      Bermuda  2016                 108747.356144
439      Bermuda  2017                 112339.425587
440      Bermuda  2018                 113204.822108
441      Bermuda  2019                 115798.040776
442      Bermuda  2020                 106976.546240
443      Bermuda  2021                 112765.081673
444      Bermuda  2022                 121613.939984




In [230]:
#  5. Create a focused dataset for 2023 analysis

df_2023 = df_clean[df_clean['Year'] == 2023].copy()


In [231]:
# For countries missing 2023 data, use the most recent available data

recent_data = df_clean.sort_values(['Country Name', 'Year']).groupby('Country Name').last().reset_index()
df_final = recent_data.copy()

print("=== FINAL DATASET STATS ===")
print(f"Final countries with data: {df_final['Country Name'].nunique()}")
print(f"GDP data coverage: {df_final['GDP per capita (current US$)'].notna().sum()}/{len(df_final)}")
print(f"Internet data coverage: {df_final['Individuals using the Internet (% of population)'].notna().sum()}/{len(df_final)}")

=== FINAL DATASET STATS ===
Final countries with data: 265
GDP data coverage: 262/265
Internet data coverage: 215/265


In [232]:
# 6. Create additional derived variables for analysis

df_final['GDP_category'] = pd.cut(df_final['GDP per capita (current US$)'], 
                                 bins=[0, 1000, 4000, 12000, 25000, float('inf')],
                                 labels=['Low', 'Lower-middle', 'Middle', 'Upper-middle', 'High'])

df_final['Internet_category'] = pd.cut(df_final['Individuals using the Internet (% of population)'].fillna(0),
                                     bins=[0, 20, 50, 80, 100],
                                     labels=['Low', 'Medium', 'High', 'Very High'])


In [233]:
# 7. Prepare data for mapping

# Create a clean mapping for choropleth
choropleth_data = df_final[['Country Code', 'GDP per capita (current US$)']].dropna()
choropleth_data = choropleth_data.set_index('Country Code')['GDP per capita (current US$)'].to_dict()

internet_data = df_final[['Country Code', 'Individuals using the Internet (% of population)']].dropna()
internet_data = internet_data.set_index('Country Code')['Individuals using the Internet (% of population)'].to_dict()

print(f"Countries with GDP data for mapping: {len(choropleth_data)}")
print(f"Countries with Internet data for mapping: {len(internet_data)}")


Countries with GDP data for mapping: 262
Countries with Internet data for mapping: 215


## Step 4: Quality Data Checks

In [234]:
print("=== DATA QUALITY CHECKS ===")

# Check for duplicate country-year combinations
duplicates = df_clean.duplicated(subset=['Country Name', 'Year'])
print(f"Duplicate country-year entries: {duplicates.sum()}")


=== DATA QUALITY CHECKS ===
Duplicate country-year entries: 0


In [235]:
# Check data consistency across years for sample countries

sample_countries = ['United States', 'China', 'India', 'Brazil', 'Nigeria']
for country in sample_countries:
    country_data = df_clean[df_clean['Country Name'] == country]
    if not country_data.empty:
        gdp_growth = country_data['GDP per capita (current US$)'].pct_change().mean() * 100
        print(f"{country}: Average GDP growth: {gdp_growth:.2f}%")

United States: Average GDP growth: 5.28%
China: Average GDP growth: 25.44%
India: Average GDP growth: 12.89%
Brazil: Average GDP growth: 10.75%
Nigeria: Average GDP growth: 7.50%


In [236]:
# Check for data completeness by region (using World Bank regions as example)
# This would require additional region data, but we can analyze by country groups present

country_groups = ['Africa Eastern and Southern', 'Africa Western and Central', 
                 'Arab World', 'Caribbean small states', 'Central Europe and the Baltics']
regional_data = df_clean[df_clean['Country Name'].isin(country_groups)]
print(f"Regional data entries: {len(regional_data)}")

Regional data entries: 90


In [237]:
# Validate data ranges

print("\n=== DATA VALIDATION ===")
print(f"GDP range: ${df_clean['GDP per capita (current US$)'].min():.2f} - ${df_clean['GDP per capita (current US$)'].max():.2f}")
print(f"Internet usage range: {df_clean['Individuals using the Internet (% of population)'].min():.1f}% - {df_clean['Individuals using the Internet (% of population)'].max():.1f}%")
print(f"Life expectancy range: {df_clean['Life expectancy at birth, total (years)'].min():.1f} - {df_clean['Life expectancy at birth, total (years)'].max():.1f} years")


=== DATA VALIDATION ===
GDP range: $109.59 - $256580.52
Internet usage range: 0.0% - 100.0%
Life expectancy range: 18.8 - 86.4 years


## Step 5: Prepare Data for Mapping

In [238]:
# Create a mapping from country names to ISO codes (we'll use Country Code from our data)

country_code_mapping = df_2023_clean.set_index('Country Name')['Country Code'].to_dict()

In [239]:
# Prepare data for choropleth - we need ISO code and value

choropleth_data = df_2023_clean[['Country Code', 'GDP per capita (current US$)']].dropna()
choropleth_data = choropleth_data.set_index('Country Code')['GDP per capita (current US$)'].to_dict()

print(f"Countries with GDP data: {len(choropleth_data)}")

Countries with GDP data: 249


## Step 6: Create the Choropleth Map

In [240]:
# Create a base map

m = folium.Map(location=[20, 0], zoom_start=2, tiles='cartodbpositron')

In [241]:
# Create choropleth

choropleth = Choropleth(
    geo_data=world_geojson,
    data=choropleth_data,
    columns=['id', 'value'],  # GeoJSON uses 'id' for country codes
    key_on='feature.id',
    fill_color='YlGnBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='GDP per capita (current US$, 2023)',
    nan_fill_color='white',
    name='GDP Choropleth').add_to(m)

# Add tooltip
folium.GeoJsonTooltip(['name']).add_to(choropleth.geojson)

# Add layer control
folium.LayerControl().add_to(m)

# Display the map
m

In [242]:
# Save to HTML file

m.save('gdp_choropleth_map.html')

In [243]:
# Prepare data for bubble map showing GDP and Internet usage

df_2023_bubble = df_2023_clean.dropna(subset=['GDP per capita (current US$)', 'Individuals using the Internet (% of population)'])

# Create a new map
m2 = folium.Map(location=[20, 0], zoom_start=2)

# Add choropleth for GDP
Choropleth(
    geo_data=world_geojson,
    data=choropleth_data,
    columns=['id', 'value'],
    key_on='feature.id',
    fill_color='YlGnBu',
    fill_opacity=0.6,
    line_opacity=0.2,
    legend_name='GDP per capita (US$, 2023)',
    name='GDP'
).add_to(m2)

# Add bubble markers for Internet usage
for idx, row in df_2023_bubble.iterrows():
    if not pd.isna(row['GDP per capita (current US$)']) and not pd.isna(row['Individuals using the Internet (% of population)']):
        folium.CircleMarker(
            location=[row['Latitude'] if 'Latitude' in df_2023_bubble.columns else 0, 
                     row['Longitude'] if 'Longitude' in df_2023_bubble.columns else 0],
            radius=row['Individuals using the Internet (% of population)']/10,
            popup=f"{row['Country Name']}: {row['Individuals using the Internet (% of population)']}% Internet",
            color='red',
            fill=True,
            fillColor='red'
        ).add_to(m2)

# Add layer control
folium.LayerControl().add_to(m2)

m2.save('gdp_internet_comparison_map.html')
# Save to HTML file
m.save('gdp_choropleth_map.html')

## Step 7: Analysis and Discussion

What this choropleth map shows:
* The map visualizes GDP per capita across countries in 2023, revealing significant economic disparities globally. 
Wealthier nations (primarily in North America, Western Europe, and parts of Asia) show much higher GDP per capita compared to developing regions, particularly in Africa and parts of Asia.

Research Questions Answered:

* Economic disparities: The map clearly shows the vast economic inequality between developed and developing nations.

* Regional patterns: Certain regions (like Western Europe) show consistent economic development, while others (like Sub-Saharan Africa) show widespread lower economic output per capita


New Research Questions:

* What factors contribute to the economic success of countries with high GDP per capita?
* How has GDP per capita changed over time in different regions?
* What is the relationship between GDP per capita and other development indicators like internet usage or life expectancy?
* Why do some geographically similar countries have vastly different economic outcomes?


Limitations and Next Steps:

* The analysis only covers one year (2023)
* Some countries have missing data
* Future analysis could incorporate time series data to show economic development trends
* Additional variables (internet usage, life expectancy) could be layered for more comprehensive analysis
